## ДЗ по поиску

Привет! Вам надо реализивать поисковик на базе вопросов-ответов с сайта [pravoved.ru](https://pravoved.ru/questions-archive/).        
Поиск должен работать на трех технологиях:       
1. обратном индексе     
2. word2vec         
3. doc2vec      

Вы должны понять, какой метод и при каких условиях эксперимента на этом корпусе работает лучше.          
Для измерения качества поиска найдите точность (accuracy) выпадания правильного ответа на конкретный вопрос (в этой базе у каждого вопроса есть только один правильный ответ). Точность нужно измерить для всей базы.    
При этом давайте считать, что выпал правильный ответ, если он попал в **топ-5** поисковой выдачи.

> Сделайте ваш поиск максимально качественным, чтобы значение точности стремилось к 1.     
Для этого можно поэкспериментировать со следующим:       
- модель word2vec (можно брать любую из опен сорса или обучить свою)
- способ получения вектора документа через word2vec: простое среднее арифметическое или взвешивать каждый вектор в соответствии с его tf-idf      
- количество эпох у doc2vec (начинайте от 100)
- предобработка документов для обучения doc2vec (удалять / не удалять стоп-слова)
- блендинг методов поиска: соединить результаты обратного индекса и w2v, или (что проще) w2v и d2v

На это задание отведем 10 дней. Дэдлайн сдачи до полуночи 12.10.

In [44]:
import pickle

with open('qa_corpus.pkl', 'rb') as file:
    qa_corpus = pickle.load(file)

Всего в корпусе 1384 пары вопрос-ответ

In [2]:
len(qa_corpus)

1384

Первый элемент блока это вопрос, второй - ответ на него

In [4]:
qa_corpus[0]

['\nДобрый день.Мой сын гражданин Украины (ДНР),имеет вид на жительство в Р.Ф., кот.получил проживая с 2014 г. в Нижегородской области.В 2017г. переехал на постоянное место жительство в г.Ростов.Официально трудоустроился на одно из промышл.предприятий г.Ростова.Оформил временную регистрацию в Ростове.В УФМС предупредили,что по истечении 90 дней он должен либо постоянно прописаться либо покинуть территорию России.Прошу проконсультировать как быть дальше.(Вернуться домой в Донецк,но здесь идет война,работы нет.В Ростове он работает по специальности.Он инженер машиностроитель.)Временная прописка до 15 марта.  Если он сможет приобрести какую либо недвижимость,как долго будет решаться вопрос о его постоянной прописке в Ростове.Как в этом случае будет решаться вопрос с видом на жительство в Ростове? Не получится ли ,что приобретя квартиру,он не успеет в ней прописаться до окончании срока временной регистрации. С уважением Людмила Евгеньевна.\n',
 'Добрый вечер!Из Вашего вопроса вообще ничего

Файлы до моделей

In [45]:
from gensim.models import Word2Vec, KeyedVectors
import codecs
import json
w2v_path = 'models/ru.bin'
d2v_path = 'models/qa_d2v.bin'
bm25_path = 'models/qa_bm25.bin'

models = {}
vectors = {}

Предобработка базы

In [46]:
import re
import pymorphy2

w = re.compile('[A-zА-я]+')
morph = pymorphy2.MorphAnalyzer()

def get_words(text):
    words = w.findall(text)
    words = [morph.parse(w.lower())[0].normal_form for w in words]
    return words

def count_words(text):
    words = {}
    indoc = get_words(text)
    for word in indoc:
        word = word.lower()
        if word not in words:
            words[word] = 0
        words[word] = words[word] + 1
    return words

def preprocessing(docs):
    return [count_words(doc[0]) for doc in docs]

processed_qa = preprocessing(qa_corpus)
print('done preprocessing')

done preprocessing


Получение word2vec модели и базы векторов

In [47]:
def get_w2v_vector(words):
    vec = [0 for i in range(models['w2v'].vector_size)]
    size = 0
    for word, count in words.items():
        if word not in models['w2v'].wv:
            continue
        size += count
        wv = models['w2v'].wv[word]
        vec = [vec[i] + wv[i] * count for i in range(len(vec))]
    if size == 0:
        return None
    return [i / size for i in vec]
        
def get_w2v_vectors(docs):
    vectors = []
    for index, doc in enumerate(docs):
        vec = get_w2v_vector(doc)
        if vec:
            vectors.append((index, vec))
    
    return vectors

def save_w2v_base(vectors):
    cpy = []
    for vec in vectors:
        cpy.append((vec[0], [round(i, 6) for i in vec[1]]))
    file = codecs.open('qa_w2v_base.bin', 'w', 'utf-8')
    file.write(json.dumps(cpy, indent=1))
    file.flush()
    file.close()

models['w2v'] = Word2Vec.load(w2v_path)
vectors['w2v'] = get_w2v_vectors(processed_qa)
save_w2v_base(vectors['w2v'])
print('done, ' + str(len(vectors['w2v'])) + ' vectors')

done, 1384 vectors


Получение doc2vec модели и базы векторов

In [48]:
from gensim.models.doc2vec import TaggedDocument, Doc2Vec

def train_doc2vec(docs):
    tagged = []
    for index, doc in enumerate(docs):
        td = TaggedDocument(words=list(doc.keys()), tags=[index])
        tagged.append(td)

    d2v_model = Doc2Vec(vector_size=100, alpha=0.025, min_count=2, dm=1)
    d2v_model.build_vocab(tagged)
    for epoch in range(100):
        d2v_model.train(tagged,
                    total_examples=d2v_model.corpus_count,
                    epochs=d2v_model.epochs)
        # decrease the learning rate
        d2v_model.alpha -= 0.0002
        # fix the learning rate, no decay
        d2v_model.min_alpha = d2v_model.alpha
    
    d2v_model.save(d2v_path)
    return d2v_model

def get_d2v_vectors(docs):
    vectors = []
    for index, doc in enumerate(docs):
        vec = models['d2v'].infer_vector(list(doc.keys()), epochs=10)
        vec = [float(f) for f in vec]
        vectors.append((index, vec))
    return vectors

def save_d2v_base(vectors):
    cpy = []
    for vec in vectors:
        cpy.append((vec[0], [round(i, 6) for i in vec[1]]))
    file = codecs.open('qa_d2v_base.bin', 'w', 'utf-8')
    file.write(json.dumps(cpy, indent=1))
    file.flush()
    file.close()

#models['d2v'] = Doc2Vec.load(d2v_path)
models['d2v'] = train_doc2vec(processed_qa)
vectors['d2v'] = get_d2v_vectors(processed_qa)
save_d2v_base(vectors['d2v'])
print('done, ' + str(len(vectors['d2v'])) + ' vectors')

done, 1384 vectors


Получение bm25 базы

In [49]:
from bm25.query import QueryProcessor
from bm25.parse import Corpus

def build_inv_index(docs):
    corp = Corpus()
    for index, doc in enumerate(docs):
        corp.add(index, list(doc.keys()))
    
    proc = QueryProcessor(corp)
    proc.save(bm25_path)
    return proc

models['bm25'] = build_inv_index(processed_qa)
print('done')
#models['bm25'] = QueryProcessor.load(bm25_path)

done


Функции поиска по базам

In [69]:
from gensim import matutils
import numpy as np 

def similarity(v1, v2):
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    return np.dot(v1_norm, v2_norm)

def search_w2v(document):    
    vec = get_w2v_vector(count_words(document))
    results = [(i, similarity(w2v_vec[1], vec)) for i, w2v_vec in enumerate(vectors['w2v'])]
    results.sort(key = lambda x: x[1], reverse=True)
    return results

def search_d2v(document):
    vec = models['d2v'].infer_vector(get_words(document), epochs=10)
    vec = [float(f) for f in vec]
    results = [(i, similarity(d2v_vec[1], vec)) for i, d2v_vec in enumerate(vectors['d2v'])]
    results.sort(key = lambda x: x[1], reverse=True)
    return results

def search_inv_index(document):
    results = models['bm25'].run_query(get_words(document))
    results = [(int(key),val) for key, val in results.items()]
    results.sort(key = lambda x: x[1], reverse=True)
    return results

In [76]:
correct = [0,0,0,0]
max_iter = 50
for index, doc in enumerate(qa_corpus):
    if index >= max_iter:
        break
    w2v_results = search_w2v(doc[0])[0:5]
    d2v_results = search_d2v(doc[0])[0:5]
    ind_results = search_inv_index(doc[0])
    
    if len(ind_results) > 5:
        ind_results = ind_results[0:5]
    is_found = [False, False, False]
    for i in range(0, 5):
        if w2v_results[i][0] == index:
            is_found[0] = True
        if d2v_results[i][0] == index:
            is_found[1] = True
        if ind_results[i][0] == index:
            is_found[2] = True
    for i in range(0,3):
        if is_found[i]:
            correct[i] += 1
    if len([a for a in is_found if a]) > 1:
        correct[3] += 1
        
print('iterations:', max_iter)
correct = [i / max_iter * 100 for i in correct]
print('correct by w2v:', correct[0], '%')
print('correct by d2v:', correct[1], '%')
print('correct by ind:', correct[2], '%')

/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


iterations: 50
correct by w2v: 100.0 %
correct by d2v: 46.0 %
correct by ind: 100.0 %
